In [1]:
#본 내용은 kaggle Intermediate Machine Learning course(by Alexis Cook)를 기록해 놓은 것임을 밝힙니다. (https://www.kaggle.com/learn/intermediate-machine-learning)

In [2]:
from google.colab import drive 
drive.mount('/content/drive')
#구글 코랩환경 드라이브 연결

Mounted at /content/drive




1. A Simple Option: Drop Columns with Missing Values) 열을 완전히 삭제하는 방식, 잠재적으로 유용한 data를 삭제할 수 있다.
2. A Better Option: Imputation) 결측값을 어떠한 숫자로 채우는 방식 Imputation이 전부 맞진 않지만 열을 삭제하는 것보다 일반적으로 정확하다.
3. An Extension To Imputation) 결측값이 있는 행을 고려하여 Imputation 하는 방식, 경우에 따라 정확도를 개선할 수 있다.






# Example

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
#데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/ML-Study-Jam-in-DSC-Sookmyung/Intermediate Machine Learning/data/melb_data.csv')
#타켓변수 설정
y = data.Price
#숫자 예측 변수만, 단순성 유지 위해
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
#데이터를 training and validation 집합으로 각각 나눈다.
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [6]:
#필요한 라이브러리를 불러오는 부분
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 접근방식을 비교할 점수방안
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [8]:
#첫번째 방식(Drop Columns with Missing Values)
#결측값이 있는 열의 이름을 가져오는 부분
cols_with_missing=[col for col in X_train.columns if X_train[col].isnull().any()]
#training and validation data의 column들을 삭제하는 부분
reduce_X_train=X_train.drop(cols_with_missing, axis=1)
reduce_X_valid=X_valid.drop(cols_with_missing, axis=1)

print("MAE frop Approach 1 (Drop colums with missing values):")
print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))

MAE frop Approach 1 (Drop colums with missing values):
183550.22137772635


In [9]:
#두번째 방식(Imputation)
from sklearn.impute import SimpleImputer
#Imputation
my_imputer=SimpleImputer()
imputed_X_train=pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid=pd.DataFrame(my_imputer.transform(X_valid))
#Imputation 열이름을 제거하고 다시 저장하는 부분
imputed_X_train.columns=X_train.columns
imputed_X_valid.columns=X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


In [10]:
#세번째 방식(An Extension to Imputation)
#원본 데이터 변화를 막기위해 복제하는 부분
X_train_plus=X_train.copy()
X_valid_plus=X_valid.copy()
#Imputation을 포함함을 나타내는 새 열 만들기
for col in cols_with_missing:
  X_train_plus[col+'_was_missing']=X_train_plus[col].isnull()
  X_valid_plus[col+'_was_missing']=X_valid_plus[col].isnull()
#Imputation
my_imputer=SimpleImputer()
imputed_X_train_plus=pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus=pd.DataFrame(my_imputer.transform(X_valid_plus))
# Imputation 열이름을 제거하고 다시 저장하는 부분
imputed_X_train_plus.columns=X_train_plus.columns
imputed_X_valid_plus.columns=X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


왜 imputation 방식이 dropping the columns 방식보다 더 나은 결과를 보여준 것인가?

In [11]:
print(X_train.shape)

missing_val_count_by_column=(X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column>0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


3개의 column에서 결측값이 있었는데 그중 2개가 5000개가 넘는 결측값이 있어서 유용한 정보를 많이 버린것이 되었다.